In [1]:
import pymongo as pm
import mysql.connector as mdb
import requests as rq
import ujson as js
import pandas as pd
import numpy as np
from datetime import datetime as dt
from threading import Thread
from time import sleep

from utils.Container import Container

In [2]:
class CharState:
    _urls = Container(
        location='https://esi.evetech.net/latest/characters/{char_id}/location/',
        online='https://esi.evetech.net/latest/characters/{char_id}/online/',
        ship='https://esi.evetech.net/latest/characters/{char_id}/ship/'
    )
    _args = Container(
        headers=Container(
            accept='application/json',
            Authorization='Bearer {token}'
        ),
        params=Container(datasource='tranquility')
    )
    
    def __init__(self, char_id,
                 sleep_time=1, url_retrys=5,
                 mongo_login_path='./../../settings/mongo_login.json',
                 maria_login_path='./../../settings/maria_login.json'):
        self.char_id = char_id
        self._build_conns(mongo_login_path, maria_login_path)
        self._data = Container()
        self._threads = Container()
        self.sleep_time = sleep_time
        self.url_retrys = url_retrys
        
        self._token = Container(
            db='NewEdenAnalytics',
            coll='eve_characters',
            value=None,
            expires=None
        )
        
        now_time = dt.utcnow()
        self.refresh = Container()
        for key in self._urls: self.refresh[key] = now_time
            
    @property
    def args(self):
        args = self._args._dict
        args['headers']['Authorization'] = args['headers']['Authorization'].format(token=self._token.value)
        return args
    
    def start_threads(self):
        self._start_token_thread()
        self._start_data_thread()
            
    def _start_data_thread(self):
        self._threads.data = Thread(target=self._load_data_thread, daemon=True)
        self._threads.data.start()
        
    def _start_token_thread(self):
        self._threads.token = Thread(target=self._refresh_token_thread, daemon=True)
        self._threads.token.start()
    
    def _build_conns(self, mongo_login_path, maria_login_path):
        with open(mongo_login_path) as file_mongo, \
             open(maria_login_path) as file_maria:
            self.conn = Container(
                mongo=pm.MongoClient(**js.load(file_mongo)),
                maria=mdb.connect(**js.load(file_maria)),
                url=rq.Session()
            )
        
    def refresh_token(self):
        token_data = self.conn.mongo[self._token.db][self._token.coll].find_one(
            {'_id':self.char_id},
            {'tokens.access_token':1, 'tokens.expires_on':1}
        )['tokens']
        self._token.value = token_data['access_token']
        self._token.expires = token_data['expires_on']
        
    def _token_thread(self):
        while True:
            self.refresh_token()
            sleep_time = ((self._token.expires - dt.utcnow()).seconds / 60) - 2
            sleep(sleep_time)
        
    def _data_thread(self):
        while True:
            for key in self._urls:
                self.get_data(key)
            sleep(self.sleep_time)
    
    def _call_url(self, method, url, args):
        for _ in range(self.url_retrys):
            resp = rq.request(method, url, **args)
            if resp.status_code == 200: break
            sleep(0.01)
        if resp.status_code == 200:
            return resp
        else:
            return None
        
    def get_data(self, data_key):
        if dt.utcnow() > self.refresh[data_key]:
            resp = self._call_url('get', self._urls[data_key].format(char_id=self.char_id), self.args)
            if resp is not None:
                self.refresh[data_key] = dt.strptime(resp.headers['Expires'], '%a, %d %b %Y %H:%M:%S %Z')
                for key, val in js.loads(resp.content).items():
                    self._data[key] = val

In [3]:
char_state = CharState(144981811)

In [4]:
char_state.start_threads()

In [8]:
char_state._data._dict

{'online': True,
 'last_login': '2019-10-01T22:58:35Z',
 'ship_item_id': 1031127222320,
 'last_logout': '2019-10-01T13:17:34Z',
 'ship_type_id': 33468,
 'logins': 6369,
 'ship_name': 'Streamslip',
 'solar_system_id': 31001422}

In [9]:
char_state._threads._dict

{'data': <Thread(Thread-5, started daemon 139996165043968)>,
 'token': <Thread(Thread-4, started daemon 139996173436672)>}